In [ ]:
#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

# Hello world with KubeFlow Pipelines 

Welcome to your first step with KubeFlow Pipelines (KFP). This exercises focusses on: 

* Creating a KubeFlow pipeline using the KFP SDK
* Creating your first experiment and submitting the pipeline to KFP run time enviroment using the SDK 

Run this notebook on your Jupyter Hub environment on Kubeflow

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/

## Setup

Change the following constants in the code to make sure it maches your project + bucket: 

- PROJECT_NAME
- OUTPUT_DIR

In [2]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'Hellow world!-v1'
PROJECT_NAME =  'your-gcp-project-name' #'your-gcp-project-name'
OUTPUT_DIR = 'gs://path-to-your-gcs-bucket' # 'path-to-your-gcp'
BASE_IMAGE='tensorflow/tensorflow:1.11.0-py3'  # Based image used in various steps of the pipeline
TARGET_IMAGE='gcr.io/%s/pusher:latest' % PROJECT_NAME # Target image that will include our final code

In [1]:
# Let's import the libraries needed (leave as is)
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret
from kubernetes import client as k8s_client
from kfp import compiler
from kfp import notebook
from kfp import components as comp

## Create a function that we will turn in a component

Here you need to implement a python function that takes two arguments, uses Numpy to multiply the two arguments and then returns the results. Later we will use this function to create a KFP component. 

In [ ]:
@dsl.python_component(
    name='name', # give your component op a name
    description='description', #give your component a name
    base_image=BASE_IMAGE  # note you can define the base image here, or during build time. 
    )


In [ ]:
# simpel test your function

a = 2
b = 4

z = computation(a, b)

print (z)

## Build a Pipeline Step With the Above Function

#### Option One: Specify the dependency directly

First we create our component using the python function from abbove. Build_component automatically builds a container image for the component_func based on the base_image and pushes to the target_image.

In [5]:
# Build Python Component
from kfp import compiler

mult_opp = compiler.build_python_component(
          component_func= "set to your function name", # here we refer to function we created
          staging_gcs_path=OUTPUT_DIR, # staging directory
          dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
          base_image=BASE_IMAGE, # specify base image
          target_image=TARGET_IMAGE # target image
          )

2019-07-23 10:14:31:INFO:Build an image that is based on tensorflow/tensorflow:1.11.0-py3 and push the image to gcr.io/erwinh-mldemo/pusher:latest
2019-07-23 10:14:31:INFO:Checking path: gs://kfp-app/tmp...
2019-07-23 10:14:31:INFO:Generate entrypoint and serialization codes.
2019-07-23 10:14:31:INFO:Generate build files.
2019-07-23 10:14:31:INFO:Start a kaniko job for build.
2019-07-23 10:14:31:INFO:Cannot Find local kubernetes config. Trying in-cluster config.
2019-07-23 10:14:31:INFO:Initialized with in-cluster config.
2019-07-23 10:14:36:INFO:5 seconds: waiting for job to complete
2019-07-23 10:14:41:INFO:10 seconds: waiting for job to complete
2019-07-23 10:14:46:INFO:15 seconds: waiting for job to complete
2019-07-23 10:14:51:INFO:20 seconds: waiting for job to complete
2019-07-23 10:14:56:INFO:25 seconds: waiting for job to complete
2019-07-23 10:15:01:INFO:30 seconds: waiting for job to complete
2019-07-23 10:15:06:INFO:35 seconds: waiting for job to complete
2019-07-23 10:15:1

## Build a pipeline using this component

Now we can create our own pipeline pipeline. Implement a function that takes three arguments and performs three tasks:

- add_task_1 -> a * b 
- add_task_2 -> a * c
- add_task_total -> output_add_task_1 * output_add_task_2

Some tips/hints :)

- Use the opp that you just created (multi_opp)
- You can pass pipeline paramater as arguments
- You can create a dependency between tasks
- If your stuck you can look at the exmaple1 Notebook in the repo 

In [50]:
import kfp.dsl as dsl

@dsl.pipeline(
   name='Numpy multiply pipeline',
   description='A toy pipeline that performs numpy calculations.'
   )


### Compile the pipeline

We have created a component using from a Python function. Then we created our pipeline using the component. Now we can take the pipeline and compolie it. 

In [51]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

Have a look at your local workdir. Here you will find the pipeline.zip file. This zip file contains the yaml file that descirbes your pipeline. 

### Create experiment

Now we need to create an experiment. This is a workspace where you can try different configurations of your pipelines. 

In [30]:
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

### Submit Pipeline

In [53]:
import time

#Specify pipeline argument values
arguments = {'a': '2', 'b': '4', 'c': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run-%s' % (int(time.time()))
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working

The output of every step in your pipeline should look like this:

- add_task_1 -> 8 
- add_task_2 -> 16
- add_task_total -> 128

The great thing about this setup is that we can change the arguments taken by the pipeline and  re-run. Try it! 